In [26]:
import requests
import pandas as pd

url = "https://opendata-ajuntament.barcelona.cat/data/api/3/action/datastore_search"
params = {
    "resource_id": "540e48d8-c432-43df-b3ba-a0cf009b90ef",  
    "limit": 1000
}
r = requests.get(url, params=params)
df = pd.DataFrame(r.json()["result"]["records"])
df

,Superfície Residencial (ha),Codi_Districte,Densitat neta (hab/ha),Nom_Districte,Nom_Barri,Densitat (hab/ha),Codi_Barri,Població,_id,Superfície (ha),Any
0,49.6,1,952,Ciutat Vella,el Raval,429,1,47228,1,110.0,2021
1,34.1,1,639,Ciutat Vella,el Barri Gòtic,268,2,21827,2,81.6,2021
2,13.5,1,1081,Ciutat Vella,la Barceloneta,124,3,14643,3,117.9,2021
3,32.4,1,683,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",199,4,22131,4,111.0,2021
4,33.6,2,930,Eixample,el Fort Pienc,336,5,31262,5,92.9,2021
...,...,...,...,...,...,...,...,...,...,...,...
68,24.0,10,556,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,109,69,13323,69,122.7,2021
69,28.1,10,913,Sant Martí,el Besòs i el Maresme,212,70,25657,70,120.9,2021
70,15.3,10,1368,Sant Martí,Provençals del Poblenou,193,71,20949,71,108.6,2021
71,23.4,10,1111,Sant Martí,Sant Martí de Provençals,355,72,26032,72,73.4,2021


In [55]:
import requests
import pandas as pd

def fetch_dataset(resource_id, limit=1000):
    url = "https://opendata-ajuntament.barcelona.cat/data/api/3/action/datastore_search"
    res = requests.get(url, params={"resource_id": resource_id, "limit": limit})
    res.raise_for_status()
    return pd.DataFrame(res.json()["result"]["records"])


resource_ids = {
    "density": "540e48d8-c432-43df-b3ba-a0cf009b90ef",
    "income": "33ca69d2-6aef-4bbd-b4e9-3ff5d1ba5f3a"
}

# Fetch data
df_density = fetch_dataset(resource_ids["density"])
df_income = fetch_dataset(resource_ids["income"])

# Rename and clean columns for density
df_density.rename(columns={
    "Nom_Barri": "Neighbourhood",
    "Densitat (hab/ha)": "Density"
}, inplace=True)
df_density["Neighbourhood"] = df_density["Neighbourhood"].str.strip().str.lower()

# Rename and clean columns for income
df_income.rename(columns={
    "Nom_Barri": "Neighbourhood",
    "Import_Euros": "Income"
}, inplace=True)
df_income["Neighbourhood"] = df_income["Neighbourhood"].str.strip().str.lower()

# Merge on Neighbourhood only
df_merged = pd.merge(
    df_density[["Neighbourhood", "Density"]],
    df_income[["Neighbourhood", "Income"]],
    on="Neighbourhood",
    how="inner"
)
df_merged

,Neighbourhood,Density,Income
0,el raval,429,12551
1,el raval,429,10715
2,el raval,429,9937
3,el raval,429,12214
4,el raval,429,10296
...,...,...,...
930,el poblenou,217,16654
931,diagonal mar i el front marítim del poblenou,109,19707
932,diagonal mar i el front marítim del poblenou,109,22509
933,diagonal mar i el front marítim del poblenou,109,22874


In [59]:
import requests
import pandas as pd

def fetch_dataset(resource_id, limit=1000):
    url = "https://opendata-ajuntament.barcelona.cat/data/api/3/action/datastore_search"
    res = requests.get(url, params={"resource_id": resource_id, "limit": limit})
    res.raise_for_status()
    return pd.DataFrame(res.json()["result"]["records"])

# Dataset resource IDs
resource_ids = {
    "density": "540e48d8-c432-43df-b3ba-a0cf009b90ef",
    "income": "33ca69d2-6aef-4bbd-b4e9-3ff5d1ba5f3a"
}

# Fetch datasets
df_density = fetch_dataset(resource_ids["density"])
df_income = fetch_dataset(resource_ids["income"])

# Clean + rename
df_density.rename(columns={
    "Nom_Barri": "Neighbourhood",
    "Densitat (hab/ha)": "Density"
}, inplace=True)
df_density["Neighbourhood"] = df_density["Neighbourhood"].str.strip().str.lower()
df_density["Density"] = pd.to_numeric(df_density["Density"], errors="coerce")

df_income.rename(columns={
    "Nom_Barri": "Neighbourhood",
    "Import_Euros": "Income"
}, inplace=True)
df_income["Neighbourhood"] = df_income["Neighbourhood"].str.strip().str.lower()
df_income["Income"] = pd.to_numeric(df_income["Income"], errors="coerce")

# Group income (averaging it)
df_income_avg = df_income.groupby("Neighbourhood", as_index=False).mean(numeric_only=True)

# Merge: one row per Neighbourhood
df_final = pd.merge(df_density[["Neighbourhood", "Density"]], df_income_avg, on="Neighbourhood", how="inner")

# Display final result
pd.set_option("display.max_columns", None)
df_final

,Neighbourhood,Density,Income,_id
0,el raval,429,11045.714286,11.0
1,el barri gòtic,268,14032.444444,26.0
2,la barceloneta,124,14063.454545,36.0
3,"sant pere, santa caterina i la ribera",199,15395.615385,48.0
4,el fort pienc,336,19193.300000,64.5
...,...,...,...,...
61,el clot,381,17437.500000,954.5
62,el parc i la llacuna del poblenou,139,18723.666667,967.0
63,la vila olímpica del poblenou,100,25614.400000,974.0
64,el poblenou,217,18686.700000,986.5


In [69]:
import praw
from collections import Counter
import pandas as pd

# Reddit API credentials
reddit = praw.Reddit(
    client_id="r5S_4rTGRe_-l_zi5eef8A",
    client_secret="CPSED1enLWb_NfTrgo49AA9vYNILHg",
    user_agent="food-analyzer-script by u/DifferentCell8738"
)

reddit.read_only = True

# List of foods to track
foods = ["japanese", "american", "mexican", "italian", "korean", "chinese", "bbq", "spanish", "catalan",]
food_counts = Counter({food: 0 for food in foods})

# Fetch posts from r/food (can also use .top() or .new())
for post in reddit.subreddit("food").hot(limit=1000):  # you can increase to 1000 if needed
    content = (post.title + " " + post.selftext).lower()
    for food in foods:
        if food in content:
            food_counts[food] += 1

# Display result
df_food_country = pd.DataFrame(food_counts.items(), columns=["Food", "Mentions"])
df_food_country = df_food_country.sort_values(by="Mentions", ascending=False)
df_food_country.to_csv("food_co.csv", index=False)

       Food  Mentions
4    korean        12
6       bbq        11
2   mexican         8
5   chinese         8
3   italian         7
0  japanese         5
1  american         5
7   spanish         0
8   catalan         0


In [71]:
import praw
from collections import Counter
import pandas as pd

# Reddit API credentials
reddit = praw.Reddit(
    client_id="r5S_4rTGRe_-l_zi5eef8A",
    client_secret="CPSED1enLWb_NfTrgo49AA9vYNILHg",
    user_agent="food-analyzer-script by u/DifferentCell8738"
)

reddit.read_only = True

# List of foods to track
foods = ["korean bbq", "sushi", "paella", "tapas", "pasta", "pizza", "ramen", "steak", "burger",]
food_counts = Counter({food: 0 for food in foods})

# Fetch posts from r/food (can also use .top() or .new())
for post in reddit.subreddit("food").hot(limit=1000):  # you can increase to 1000 if needed
    content = (post.title + " " + post.selftext).lower()
    for food in foods:
        if food in content:
            food_counts[food] += 1

# Display result
df_food_counts = pd.DataFrame(food_counts.items(), columns=["Food", "Mentions"])
df_food_counts = df_food_counts.sort_values(by="Mentions", ascending=False)


print(df_food_counts)
df_food_counts.to_csv("food_counts.csv", index=False)

         Food  Mentions
7       steak        37
5       pizza        36
8      burger        26
4       pasta        21
6       ramen        14
1       sushi         8
2      paella         3
0  korean bbq         0
3       tapas         0


In [72]:
import os
print("food_counts.csv" in os.listdir())

True


In [73]:
import os
print("Saved to:", os.path.abspath("food_counts.csv"))

Saved to: /Users/levinschily/Desktop/IRONHACK/Week4/Project_Group/first_project/food_counts.csv
